# TD learning+价值函数估计

## I. <font color=blue>价值函数估计法的出发点：</font>

- 到目前为止，前面的方法都将state or action value处理成table，如sarsa、Q-learning都是在估计table中每个位置的state-action对应的action value。<font color=norange>所以他们又称为tabular Sarsa和tabular Q-learning。</font>这些方法仅适用于state-action数量很少的场景。当state-action数量很大时，算法每次迭代只能处理table中单个元素，效率低。

- <font color=green>**一种改进思路**</font>：function approximation methods。指的是用函数代替table，通过求解函数中的参数来找到state-action与value之间的稳定关系。
$$\begin{align}
V^{\pi}(s) & = V_{\theta }^{\pi}(s) = f(s,\theta )\\
Q^{\pi}(s, a) & = Q_{\theta }^{\pi}(s, a) = h(s, a, \theta ) \\
\end{align}$$
  - f(·)和h(·)可以是各种mapping关系，包括神经网络。问题求解的目标不再是直接求V(s)和Q(s,a)，而转变成了求函数的参数$\theta$。
  - 提升算法效率的方式：参数的一次调整会影响所有state-action pair的价值，算法的优化效率更高
  - 代价是此时函数只是原价值函数的approximation，如果函数的表达力不够，那么不论参数如何优化都不能准确表达原价值函数。

- <font color=green>**方法调整**</font>：
  - sarsa和Q-learning等基于RM的TD算法求解的目标是价值函数Q(s, a)。而用function approximation之后，求解的目标变成给定函数求参数$\theta$。
  - 目标问题变化：和前面章节一样，先用state value function来说明思路。
$$\begin{align}
原问题：& V^{\pi}(S)\\
新问题：\\
& \underset{\theta}{min}J(\theta) 
 = \underset{\theta}{min}E\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )^2  \\
\Leftrightarrow  \ \ &\nabla_{\theta }E\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )^2  =0\\
\Leftrightarrow  \ \ &E\nabla_{\theta }\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )^2=0\\
\Leftrightarrow  \ \ &E\left [\left ( V^{\pi}(S)-\hat{V}(S,\theta ) \right )\nabla_{\theta }\hat{V}(S,\theta )\right ]=0
\end{align}
$$
  - 仍然是期望形式，因此还是可以用RM算法迭代求解，对应RM的第二种典型应用：SGD形式
  - **由于没有$V^{\pi}$的真实值的样本，所以不能直接用RM算法，和上一章中TD learning的思路一样，改用当前时刻对$V^{\pi}$的‘最佳估计’。**

- <font color=red>**代价：TD learning加上function approximation之后，除非特定条件下，否则不再具备almost surely convergence的性质。**</font>

- <font color=blue>这部分是后面算法的基础，分析在什么条件下，MDP的状态s的概率分布是稳定的，也就是P(s)稳定。从而可以用传统的SGD等方法求：
$$\begin{align}
\underset{w}{min}J(w) 
& = \underset{w}{min}E\left [\left ( v^{\pi}(S)-\hat{v}(S,w) \right )^2 \right ] \\
& = \underset{w}{min}\sum _{s\in \mathcal{S}}P(s)[v^{\pi}(s)-\hat{v}(s,w)]^2 \\
\end{align}$$</font>

### I.1 MDP条件下的状态转移概率矩阵

1. 定义状态转移概率矩阵: 假设每一步都有n种状态。
   - (1)单步状态转移概率矩阵：$P^{\pi}\in \mathbb{R}^{n\times n} $，其中单个元素为$[P^{\pi }]_{ij}$，表示当前状态是$s_i$时，下一步状态为$s_j$的概率。
$$[P^{\pi }]_{ij}=p^{\pi}(s_j|s_i)=\sum_{a\in \mathcal{A}}p(s_j|s_i, a)\pi(a|s_i)$$
<font color=blue>**$P^{\pi}$由环境严格说是transition function$p(s_j|s_i, a)$和策略$\pi$共同决定，不受其他因素影响。**</font> 
   - (2)k步状态转移概率矩阵：$P^{\pi,(k)}\in \mathbb{R}^{n\times n}$，简记为$P^{(k)}$，其中单个元素$[P^{\pi,(k)}]_{ij}$（简记为$[P^{(k)}]_{ij}$）表示在策略$\pi$下，状态从$s_i$经过k步转移到$s_j$的概率。
$$
\begin{align}
[P^{\pi,(k)}]_{ij} & = [P^{(k)}]_{ij} = p^{\pi}(S_{t_k}=s_j|S_{t_0}=s_i) \\
[P^{\pi }]_{ij} & = [P^{(1)}]_{ij}
\end{align}
$$

2. **多步状态转移的概率可以表示为多个单步状态转移概率矩阵相乘。** 
   - 根据经定义，两步时有：
$$
[(P^{\pi})^2]_{i, j}  = [P^{\pi}\times P^{\pi}]_{i, j}  = \sum_{h=1}^n [P^{\pi}]_{ih}\times [P^{\pi}]_{hj} = [P^{(2)}]_{i,j} 
$$
   - 三步时有：
$$
\begin{align}
[(P^{\pi})^3]_{i, j} 
& = [(P^{\pi})^2\times P^{\pi}]_{i, j} \\
& = \sum_{h_2=1}^n \underset{p(s_i\rightarrow\forall s\rightarrow s_{h2})}{\underbrace{[(P^{\pi})^2]_{ih_2}}}\times [P^{\pi}]_{h_2j}  \\
& = \sum_{h_2=1}^n \underset{p(2步从s_i到s_{h2})}{\underbrace{[P^{(2)}]_{ih_2}}}\times  \underset{p(s_{h2}\rightarrow s_j)}{\underbrace{[P^{\pi}]_{h_2j}}}  \\
& =  \underset{p(3步从s_i\rightarrow s_j)}{\underbrace{[P^{(3)}]_{ij}}}\\
\end{align}
$$
   - 扩展到多步：
$$
[(P^{\pi})^k]=[\underset{k个P^{\pi}相乘}{\underbrace{P^{\pi}\times P^{\pi}\times ... \times P^{\pi}}}  ]_{i, j}
= [P^{(k)}]_{i,j}
$$

### I.2 MDP条件下的状态分布定义

1. $d_0\in \mathbb{R}^n$是n维向量，第i个元素$d_0(s_i)$表示起点时刻处于状态$s_i$的概率。

2. $d_k(s_i)$表示从起点开始，经过k步之后，agent处于状态$s_i$的概率。
$$
d_k = \begin{pmatrix}
d_k(s_1) \\
d_k(s_2) \\
... \\
d_k(s_n)
\end{pmatrix}, 且\sum_{s\in S}d_k(s)=1 
$$

3. 有性质：
$$
\begin{align}
d_k(s_i) & = \sum _{h = 1}^nd_0(s_h)[P^{(k)}]_{hi} \ ,\ i = 1, 2, ..., n \\
矩阵表达方式： \\
d_k^T & = d_0^TP^{(k)}
\end{align}
$$

4. 状态分布的变化取决于状态转移概率
$$
d_k^T=d_{k-1}^TP^{(1)}=d_{k-1}^TP^{\pi}
$$
<font color=green>因为$P^{\pi}$由环境$p(s_j|s_i, a)$和策略$\pi$共同决定，不受其他因素影响。所以，$d_k^T$也由环境、策略决定，但可能会受初始时刻的$d_0$影响。</font>

### I.3 MDP条件下状态分布的收敛性

#### I.3.1 状态转移矩阵可能具有的性质

1. **accessible**: 如果$[P^{\pi}]^k_{ij}>0，且k是有限的，$则称从状态si到状态sj是accessible的。
2. **communicate**: 如果从状态si到状态sj是accessible的，且反过来从sj到si也是accessible的，那么就说si和sj是communicate的。

3. **irreducible**: 如果MDP过程中所有的状态pair都是communicate的，那么就称该马尔科夫过程是irreducible的。
   - 数学定义：对任意$i\ne j$，都存在$k\ge 1$使得$[P^k]_{ij}>0$
   - 实践含义：任意状态i都能在有限步之内到达任意其他状态j
   - **<font color=purple>一般而言，exploratory policies比如$\epsilon-greedy$策略能够让马尔科夫过程具备irreducible特点。</font>** 
     - <font color=orange>说明：因为exploratory policies在任意时刻选择任意action的概率都大于0。只要transition function允许采用了不同action之后，任意状态都是accessible的，那么马尔科夫过程的所有状态pair都是communicate的。也因此，该马尔科夫过程是irreducible的。</font>

4. **regular**: 如果存在$k\ge 1$使得对任意i，j都满足$[P^k]_{ij}>0$，则称该马尔科夫过程是regular的。
   - 数学定义：存在k，使得$P^{\pi,(k)}>0$
   - 实践含义：最多用k步，任意状态i都能到达任意其他状态j
   - <font color=orange>regular是比irreducible更强的条件：irreducible只要求对于任意$s_i和s_j$，都能找到一个k，满足$[P^k]_{ij}>0$。因此每个状态pair找到的k值可以不同。regular要求存在同一个k，对任意$s_i和s_j$都满足$[P^k]_{ij}>0$。</font>
$$regular \Rightarrow irreducible, regular \nLeftarrow irreducible$$

  - 如果一个马尔科夫过程是irreducible的，且存在状态 $s_i满足[p^{\pi}]_{ii}>0$ ，那么该马尔科夫过程就是regular的。此时只要取k所有state pair完成一次communicate所需步数中最大的那个，其他的可以在i状态上不停做 $s_i\rightarrow s_i$ ，就能满足regular的特征。

5. 当马尔科夫过程满足regular性质时，如果$P^{\pi,(k)}>0$，则对$\forall k^{'}> k$都满足$P^{\pi,(k^{'})}>0$。
   - <font color=grey>*简单证明：因为马尔科夫过程满足regular时，$P^{\pi}$不可能有全为0的列向量。*</font>

#### I.3.2 状态分布的第一种收敛方式：limiting distribution

1. **定义**
   - MDP条件下给定策略$\pi$的状态转移矩阵$P(^{\pi})$和初始状态分布$d_0$，如果状态分布$d_k$收敛到一个constant value，就将该值称为limiting distribution，记为 $d^{\pi}$ , 即：
$$\lim_{k \to \infty} d_k^T =\lim_{k \to \infty}d_0^T(P^{\pi })^k  = (d^{\pi})^T$$

2. **limiting distribution存在的充要条件**：<font color=brown>[证明见附录1]</font>
$$
\begin{align}
\lim_{k \to \infty} (P^{\pi })^k & = \mathbb{1}_n(d^{\pi})^T \\
&= \begin{pmatrix}
1 \\
1 \\
... \\
1
\end{pmatrix}(d^{\pi}(s_1), d^{\pi}(s_2),... ,d^{\pi}(s_n))\\
&=\begin{pmatrix}
d^{\pi}(s_1)  & d^{\pi}(s_2) & ... & d^{\pi}(s_n)\\
d^{\pi}(s_1)  & d^{\pi}(s_2) & ... & d^{\pi}(s_n)\\
...  &  &  & \\
d^{\pi}(s_1)  & d^{\pi}(s_2) & ... & d^{\pi}(s_n)
\end{pmatrix}，所有行都一样
\end{align}
$$

3. **性质1**：<font color=blue>limiting distribution由system model和策略$\pi$决定，与初始状态分布无关。</font>
   - 从定义可见，不论初始分布是多少，经过足够长的状态转移步数之后，总是收敛到相同的limiting distribution。

4. **性质2**：如果马尔科夫过程存在limiting distribution，那么该分布是唯一的。
   - 证明：可以用定义式+反证法

5. **求解$d^{\pi}$**: 
$$\begin{align}
d_k^T = d_{k-1}^T P^{\pi },& 因此\lim_{k \to \infty} d_k^T  = \lim_{k \to \infty} d_{k-1}^T P^{\pi }\\
limiting\  distribution时：\\
&(d^{\pi})^T  = (d^{\pi})^TP^{\pi }\\
等价于求解： \\
&(P^{\pi }-I)^Td^{\pi} = 0 \\
& \left [(P^{\pi})^T-I \right]d^{\pi} = 0 \\
\end{align}$$
   - 所以，$d^{\pi}$是矩阵$(P^{\pi})^T$的特征根，且对应的特征向量是1。

#### I.3.3 状态分布的第二种收敛方式：stationary distribution

1. **定义**
   - 如果状态分布满足：$d^T = d^TP^{\pi }$，就将$d$称为stationarity distribution。通常也记为$d^{\pi}$。

2. <font color=green>从limiting distribution的定义可知，limiting distribution必然是stationary distribution。但反之不然。</font>
$$
\begin{align}
反例：& 取P^{\pi}=\begin{pmatrix}
 0 &1 \\
 1 &0
\end{pmatrix}，此时有stationary\ distribution=\begin{pmatrix}
0.5 \\
0.5
\end{pmatrix} \\
& 但没有limiting\ distribution，因为：(P^{\pi})^2=P^{\pi}\times P^{\pi}=I, (P^{\pi})^3=P^{\pi},... \\
正例：
& 取P^{\pi}=\begin{pmatrix}
0.7& 0.3 \\
0.4& 0.6
\end{pmatrix}，此时有stationary\ distribution=\begin{pmatrix}
0.571 \\
0.429
\end{pmatrix} \\
& 也有limiting\ distribution=\begin{pmatrix}
0.571 \\
0.429
\end{pmatrix}，因为：(P^{\pi})^n\rightarrow \begin{pmatrix}
0.571& 0.429 \\
0.571& 0.429
\end{pmatrix}
\end{align}
$$

- **性质1**：<font color=green>$d^{\pi}$是矩阵$(P^{\pi})^T$的特征根，且对应的特征向量是1。</font> <font color=blue>**收敛后$d^{\pi}$的取值由状态转移概率矩阵$P^{\pi}$决定，与agent的初始状态无关。** </font>
$$\begin{align}
&\because (d^{\pi})^T = (d^{\pi})^TP^{\pi } \\
&\therefore 0 = (d^{\pi})^T(P^{\pi }-I) \\
等价于求解：&(P^{\pi }-I)^Td^{\pi} = \left [(P^{\pi})^T-I \right]d^{\pi} = 0 \\
\end{align}$$

- **性质2**：<font color=blue>当马尔科夫过程是regular的，且有stationary distribution，则对所有状态s都有$d^{\pi}(s)>0$。limiting distribution也是stationary distribution，所以该性质对limiting distribution也成立。</font>
      - <font color=grey>证明：
$$
\begin{align}
& 此时满足(d^{\pi })^T = (d^{\pi })^TP^{\pi}= (d^{\pi })^TP^{\pi}P^{\pi }=...=(d^{\pi })^T(P^{\pi})^k\\
& \because regular条件下\exists k>0,s.t.(P^{\pi})^k > 0\\
& 又，d^{\pi }不会所有元素都为0，所以(d^{\pi })^T(P^{\pi})^k>0 \\
& \therefore (d^{\pi})^T > 0 
\end{align}
$$
</font>

- **性质3**：和limiting distribution不同，如果马尔科夫过程有stationary distribution，那么该分布不一定唯一。
  - 说明：根据定义，stationary distribution是矩阵$(P^{\pi})^T$的特征根，且对应的特征向量是1。而只要矩阵$(P^{\pi})^T$的特征向量1对应的是多重特征根，那么此时的stationary distribution就不是唯一的。

- **性质4（stationarity distribution存在且唯一的充要条件）**：
  - 如果马尔科夫过程有唯一的stationary distribution，则该马尔科夫过程是irreducible的。
  - 状态有限的irreducible Markov Process有且仅有唯一的stationarity distribution。
     - <font color=grey>证明：利用Perron-Frobenius theorem</font>    

#### I.3.4 limit distribution与stationary distribution之间的关系

1. 收敛性：如果Markov过程满足limiting distribution，该过程一定会收敛到$d^{\pi}$。但是，当Markov过程存在stationary distribution，则不能保证MarkvoProcess经过足够长的时间后一定会收敛到$d^{\pi}$。
2. 存在性：<font color=green>**状态有限的irreducible Markov Process一定有stationarity distribution。但不一定有limiting distribution。**</font>
3. 唯一性：
   - (1) 状态有限的irreducible Markov Process必然有唯一的stationary distribution。
   - (2) 如果Markov Process存在limiting distribution，那么该分布不仅唯一，且与马尔科夫过程的初始状态无关。

### I.4 stationary distribution在RL算法中的应用

- <font color=green>**在MDP条件下，exploratory policies比如$\epsilon-greedy$策略能够让马尔科夫过程具备irreducible特点。这时候MDP有且仅有唯一的stationary distribution。也就是说，如果agent按照给定策略工作足够长的时间，那么它处于任意state的概率的分布此时就能用该stationary distribution来刻画。**</font>

- 如果要用函数方式估计$V^{\pi}(s)$，那么此时可以用stationary distribution来帮助刻画目标函数：
$$\begin{align}
J(w) 
& = E\left [\left (V^{\pi }(S)-\hat V(S, w)\right)^2 \right ] \\
& = \sum_{s\in \mathcal{S} } d^{\pi }(s)(V^{\pi }(s)-\hat V(s, w))^2
\end{align}$$

- 如果要直接计算 $d^{\pi }(s)$ 需要知道真实的概率转移矩阵 $P^{\pi}$ ，但实践中并不知道。所以通常不会直接计算stationary distribution，而是用MC或者其他基于样本的方法来做估计。但<font color=green>**stationary distribution的存在意味着s的状态分布是稳定的，为J(w)的均值估计提供了依据。**</font>

## II. 用function approximation做TD learning of $V^{\pi}(S)$

### II.1 基础方法：function approximation + SGD+MC

#### II.1.1 目标函数

$$
\begin{align}
\underset{w}{min}J(w) 
& = \underset{w}{min}E\left [\left ( v^{\pi}(S)-\hat{v}(S,w) \right )^2 \right ] \\
& = \underset{w}{min}\sum _{s\in \mathcal{S}}d^{\pi}(s)[v^{\pi}(s)-\hat{v}(s,w)]^2 \\
w & = \underset{w}{argmin}J(w) \\
  & = \underset{w}{argmin} E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ]
\end{align}
$$
- $v^{\pi}(S)$和$\hat{v}(S,w)$分别是给定策略$\pi$时，所有state value的真实值和估计值。
- 这里的期望是针对状态$s\in S$的分布而言。

#### II.1.2 求解优化问题

1. 等价目标函数:
$$J(w) = \frac{1}{2} E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ] $$
2. 梯度：
$$
\begin{align}
\nabla J(w) 
& = \frac{1}{2} \nabla_w E\left [\left ( v^{\pi}(S)-\hat{v}(S,w)\right )^2 \right ] \\
& = -E\left [v^{\pi}(S)-\hat{v}(S,w)\right  ]\nabla_w \hat{v}(S,w)
\end{align}
$$
3. SGD算法：
$$
\begin{align}
w_{t+1} &  = w_t - \alpha_t\nabla_w J(w_t) \\
&= w_t + \alpha_t\left [v^{\pi}(s_t)-\hat{v}(s_t,w_t)\right  ]\nabla_w \hat{v}(s_t,w_t)
\end{align}
$$
4. 问题：<font color=red>**由于$v^{\pi}(s_t)$未知，因此无法直接使用SGD。**</font>
5. 一种简单的解决方法：Monte Carol方法。用episode的信息来估计所有状态$s$的state value，然后代入SGD求解。
6. 引出新的问题，MC只能在抽出很多episodes之后才能计算，如果要实时处理样本，需要新的方法。

### II.2 function approximation + TD learning

- 迭代式为：$\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma \hat{v}(s_{t+1,w_t})-\hat{v}(s_t,w_t)\right  ]\nabla_w \hat{v}(s_t,w_t)}
$

## III. 用function approximation做TD learning of $Q^{\pi}(S,A)$

### III.1 Sarsa with function approximation

#### III.1.1 迭代过程推导

- 目标问题：
$$\begin{align}
&\underset{\theta}{min}J(\theta) 
 = \underset{\theta}{min}E\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2  \\
\Leftrightarrow  \ \ &\nabla_{\theta }E\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2  =0\\
\Leftrightarrow  \ \ &E\nabla_{\theta }\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2=0\\
\end{align}$$

- RM变量：
$$\begin{align}
g(\theta ) 
& = E\nabla_{\theta }\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )^2\\
& = -E\left [\left ( Q^{\pi}(S,A)-\hat{Q}(S,A,\theta ) \right )\nabla_{\theta }\hat{Q}(S,A,\theta )\right ] \\
\tilde g(\theta_t ,\eta_t ) 
& = -\left ( Q^{\pi}(s_t,a_t)-\hat{Q}(s_t,a_t,\theta_t) \right )\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t}
\end{align}$$

- RM迭代式：
$$\begin{align}
\theta_{t+1} 
& = \theta_t -\alpha_t(\theta_t)\tilde g(\theta_t ,\eta_t ) \\
& = \theta_t +\alpha_t(\theta_t)\left ( Q^{\pi}(s_t,a_t)-\hat{Q}(s_t,a_t,\theta_t) \right )\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t}
\end{align}$$

- 由于$ Q^{\pi}(s_t,a_t)$未知，和tabular TD中一样，将其替换为当前时刻的最佳估计量得到新的迭代式：
$$\begin{align}
\theta_{t+1} 
& = \theta_t +\alpha_t(\theta_t)\left [ Q^{\pi}(s_t,a_t)-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t} \\
& \rightarrow  \theta_t +\alpha_t(\theta_t)\left [ [r_{t+1}+\gamma Q^{\pi}(s_{t+1},a_{t+1})]-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t} \\
& \rightarrow \theta_t +\alpha_t(\theta_t)\left [ [r_{t+1}+\gamma \hat Q(s_{t+1},a_{t+1}, \theta _t)]-\hat{Q}(s_t,a_t,\theta_t) \right ]\nabla_{\theta }\hat{Q}(s_t,a_t,\theta )|_{\theta =\theta_t} \\
\end{align}
$$

#### III.1.2 算法伪码

- <img src="./pics/sarsaapprox.png" alt="alt text" width="560"/>


### III.2 Q-learning with function approximation

- 迭代式: $
\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma\underset{a\in \mathcal{A}}{max}\ \hat{q}(s_{t+1},a,w_t)-\hat{q}(s_t,a_t,w_t)\right  ]\nabla_w \hat{q}(s_t,a_t,w_t)}
$

## IV. Deep Q-learning

- 思想：function approximation使用deep neural network
- 迭代式：$
\color{Blue} {w_{t+1} = w_t + \alpha_t\left [r_{t+1}+\gamma\underset{a\in \mathcal{A}}{max}\ \hat{q}(s_{t+1},a,w_t)-\hat{q}(s_t,a_t,w_t)\right  ]\nabla_w \hat{q}(s_t,a_t,w_t)}$
  - 其中$q(s_t,a_t,w_t)$是DNN

- 算法伪码
  - <img src="./pics/deepqlearning.png" alt="alt text" width="560"/>


- 算法特征：
  1. off-policy：有两个网络，behavior network用来explore environment，获得训练数据。target network用于迭代求目标参数。target network中的参数每次迭代都在改变；隔一段时间behavior network中的参数更新为target network的参数，在此之前，behavior network中的参数不像target network中那样每次迭代都改变。
  2. replay buffer：存放behavior network获得的数据。每个{s, a, r, s'}构成一个样本点，behavior network得到的episodes都被拆散成单个的样本点存入replay buffer。每次迭代时都从buffer中取出一个minibatch的样本来做训练。抽样采用以(s,a)值为index的uniform sample。
     - <font color=green>这里必须拆开成单个样本点做uniform sample是因为：不拆开的话样本点之间的相关性太强，他们都是基于当前策略连续抽的episodes。拆开后做有放回抽样可以打破这里的相关性，同时当个样本可能多次被抽中也提高了data efficiency。</font>

## 附录：

### 1. limit distribution的充要性证明

- 充分性证明：
$$
\lim_{k \to \infty} d_k^T =d_0^T\lim_{k \to \infty} (P^{\pi })^k= d_0^T\mathbb{1}_n(d^{\pi})^T = (d^{\pi})^T
$$
- 必要性证明：
$$
\begin{align}
\because &\ \lim_{k \to \infty} d_k^T  = d_0^T\lim_{k \to \infty} (P^{\pi })^k， (d^{\pi})^T  = d_0^T\mathbb{1}_n(d^{\pi})^T 
\\
if\ & \lim_{k \to \infty} d_k^T  = (d^{\pi})^T\ ,\  then \ \  \ d_0^T\lim_{k \to \infty} (P^{\pi })^k  = d_0^T\mathbb{1}_n(d^{\pi})^T  \\
\Rightarrow & \ d_0^T \left [\lim_{k \to \infty} (P^{\pi })^k - \mathbb{1}_n(d^{\pi})^T\right ] =0 \\
\Rightarrow & \ \lim_{k \to \infty} (P^{\pi })^k - \mathbb{1}_n(d^{\pi})^T =0 
\end{align}
$$